In [4]:
%matplotlib widget
from ipywidgets import IntSlider
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as py
from geopy.geocoders import Nominatim

## ¿Cómo ha evolucionado la pandemia en el mundo?

In [5]:
df_worldwide_daily = pd.read_csv('./data/worldometer_coronavirus_daily_data.csv')
df_worldwide_daily['date'] = pd.to_datetime(df_worldwide_daily['date'])
df_worldwide_daily = df_worldwide_daily[df_worldwide_daily['country'].isin(df_worldwide_daily.groupby('country')[['active_cases']].sum().sort_values('active_cases', ascending=False).head(10).index)]
df_active_cases = df_worldwide_daily.groupby(['date', 'country'])[['active_cases']].sum().reset_index()
df_active_cases = df_active_cases[df_active_cases['active_cases'] > 0]
countries = df_active_cases['country'].unique()

data = []
for country in countries:
    trace = go.Scatter(x=df_active_cases[df_active_cases['country'] == country]['date'], 
                       y=df_active_cases[df_active_cases['country'] == country]['active_cases'], 
                       name=country,
                       mode='lines')
    data.append(trace)

layout = go.Layout(title='COVID-19 Active Cases by Country',
                   xaxis=dict(title='Date', tickangle=300),
                   yaxis=dict(title='Active Cases'))
fig = go.Figure(data=data, layout=layout)


fig.layout.plot_bgcolor = 'rgba(0,0,0,0)'
fig.layout.paper_bgcolor = 'rgba(0,0,0,0)'

fig.layout.xaxis.gridwidth = 0.5
fig.layout.yaxis.gridwidth = 0.5
fig.layout.xaxis.gridcolor = 'lightgrey'
fig.layout.yaxis.gridcolor = 'lightgrey'

fig.layout.xaxis.linewidth = 0.5
fig.layout.yaxis.linewidth = 0.5
fig.layout.xaxis.linecolor = 'lightgrey'
fig.layout.yaxis.linecolor = 'lightgrey'

fig.layout.xaxis.zerolinecolor = 'lightgrey'
fig.layout.yaxis.zerolinecolor = 'lightgrey'
fig.layout.xaxis.zerolinewidth = 0.5
fig.layout.yaxis.zerolinewidth = 0.5

fig.show()

fig.write_html("fig1.html")

In [6]:

country_lat_lon = {}
def get_lat_lon_by_name(name):
    if name in country_lat_lon:
        return country_lat_lon[name]
    else:
        geolocator = Nominatim(user_agent="geoapiExercises")
        location = geolocator.geocode(name)
        country_lat_lon[name] = (location.latitude, location.longitude)
        return country_lat_lon[name]

df_active_cases['month_year'] = pd.to_datetime(df_active_cases['date']).dt.strftime('%Y-%m')
df_active_cases['lat'] = df_active_cases['country'].apply(lambda x: get_lat_lon_by_name(x)[0])
df_active_cases['lon'] = df_active_cases['country'].apply(lambda x: get_lat_lon_by_name(x)[1])
df_active_cases['text'] = df_active_cases['country'] + ': ' + df_active_cases['active_cases'].astype(str) + ' active cases'

data = [go.Scattergeo(
    lon=df_active_cases['lon'],
    lat=df_active_cases['lat'],
    text=df_active_cases['text'],
    mode='markers',
    marker=dict(
        size=np.log10(df_active_cases['active_cases']),
        color='red'
    )
)]
steps = []
for i, month_year in enumerate(df_active_cases['month_year'].unique()):
    step = dict(
        method='restyle',
        args=['visible', [df_active_cases['month_year'] == month_year]],
        label=month_year
    )
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Date: "},
    pad={"t": 50},
    steps=steps
)]

layout = go.Layout(
    title='COVID-19 Active Cases by Country',
    geo=dict(
        scope='world',
        projection=dict(type='natural earth'),
        showland=True,
        landcolor='rgb(250, 250, 250)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor='rgb(217, 217, 217)',
        countrycolor='rgb(217, 217, 217)'
    ),
    updatemenus=[dict(type='buttons', showactive=False, buttons=[dict(label='Play',
                                            method='animate',
                                            args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True,mode='immediate',transition=dict(duration=300))])])],
    sliders=sliders,
    xaxis=dict(title='Date')
)

fig = go.Figure(data=data, layout=layout)

fig.layout.plot_bgcolor = 'rgba(0,0,0,0)'
fig.layout.paper_bgcolor = 'rgba(0,0,0,0)'

fig.layout.xaxis.gridwidth = 0.5
fig.layout.yaxis.gridwidth = 0.5
fig.layout.xaxis.gridcolor = 'lightgrey'
fig.layout.yaxis.gridcolor = 'lightgrey'

fig.layout.xaxis.linewidth = 0.5
fig.layout.yaxis.linewidth = 0.5
fig.layout.xaxis.linecolor = 'lightgrey'
fig.layout.yaxis.linecolor = 'lightgrey'

fig.layout.xaxis.zerolinecolor = 'lightgrey'
fig.layout.yaxis.zerolinecolor = 'lightgrey'
fig.layout.xaxis.zerolinewidth = 0.5
fig.layout.yaxis.zerolinewidth = 0.5

py.plot(fig, auto_open=False, filename='fig2.html')

fig.show()